In [1]:
import numpy as np
import pandas as pd
import re
import os
from DS_Planck_Unet import *
from DS_detector import *
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from DS_healpix_fragmentation import radec2pix

In [2]:
def get_ep(s):
    return int(re.search('ep(\d+)', s)[0][2:])

In [3]:
dirname = '/home/rt2122/Models/act_ros/'
files = list(filter(lambda x: get_ep(x) <= 20, next(os.walk(dirname))[-1]))
files[0], get_ep(files[0])

('act_ros.ep0014-vl0.035122-l0.028428.hdf5', 14)

In [4]:
step = 8

In [5]:
val_pix

[9, 38, 41]

In [ ]:
cats_dict = {}
for file in tqdm(files):
    model = load_planck_model(os.path.join(dirname, file))
    ep = get_ep(file)
    cats_dict[ep] = []
    for i in val_pix:
        all_dict = gen_pics_for_detection(i, model, step=step)
        cats_dict[ep].append(detect_clusters_connected(all_dict, thr=0.1, ipix=i))
    cats_dict[ep] = pd.concat(cats_dict[ep], ignore_index=True)
    cats_dict[ep].to_csv(
        '/home/rt2122/Data/scan_act/act_ros_val_thr0.1_ep{ep}_step{step}.csv'.format(ep=ep, 
                step=step), index=False)

/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
stat_df = pd.DataFrame({'planck_z' : [], 'planck_no_z':[], 'mcxcwp' : [], 
                        'act_erosita':[], 
                       'fp':[]})

In [ ]:
for ep in cats_dict:
    df = cats_dict[ep]
    cur_line = {}
    for cat in ['planck_z', 'planck_no_z', 'mcxcwp', 'act_erosita']:
        df_cat = df[df['catalog'] == cat]
        if len(df_cat) == 0:
            continue
        cur_line[cat] = np.count_nonzero(df_cat['status'] == 'tp') / len(df_cat)
    cur_line['fp'] = np.count_nonzero(df['status'] == 'fp')
    stat_df.loc[ep] = cur_line

In [ ]:
stat_df.sort_index(inplace=True)
stat_df

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(22, 8))
ax[0].grid(b=True, which='major', color='#666666', linestyle=':')
for cat, c in zip(['planck_z', 'planck_no_z', 'mcxcwp', 'act_erosita'], 'bgrc'):
    line, = ax[0].plot(stat_df.index, stat_df[cat], c+'o-')
    line.set_label(cat)
ax[0].set_yticks(np.arange(0, 1, 0.1))
ax[0].legend()
line, = ax[1].plot(stat_df.index, stat_df['fp'], 'ko-')
line.set_label('fp')
ax[1].legend()